In [ ]:
import numpy as np
import math
import cv2
import tdmclient.notebook
import utils

In [ ]:
await tdmclient.notebook.start()

In [ ]:
# await tdmclient.notebook.stop()

In [ ]:
# %%run_aseba
# onevent run
#     motor.left.target = 300
#     motor.right.target = 300

# onevent turnleft
#     motor.left.target = -300
#     motor.right.target = 300

# onevent turnright
#     motor.left.target = 300
#     motor.right.target = -300

# onevent stop
#     motor.left.target = 0
#     motor.right.target = 0

Define possible objects and marker colors

In [ ]:
ball_blue = [[103, 196, 130], [110, 255, 224]]
red = [[0, 106, 180], [9, 196, 229]]
brown = [[10, 61, 123], [16, 198, 167]]
green = [[31, 78, 114], [65, 199, 193]]
yellow = [[23, 170, 154], [31, 255, 248]]
blue = [[75, 116, 167], [100, 255, 255]]

In [ ]:

robot1 = {"front":1, "back":2, "f_hsv": red, "b_hsv": green, "front_loc":(), "back_loc":()}
# robot2 = {"front":3, "back":4, "f_hsv": [[], []], "b_hsv": [[], []], "front_loc":(), "back_loc":()}
ball = {"center":5, "hsv": ball_blue, "loc":()}
goal1 = {"center": 7, "hsv": yellow, "loc":()}
goal2 = {"center": 6, "hsv": blue, "loc":()}
stadium = {"top_left_hsv": brown, "loc": []}

In [ ]:
field = {
    "ball": ball,
    "goal1": goal1,
    "goal2": goal2,
}

Define what robots are in the game

In [ ]:
robots = {
    "robot1": robot1,
    # "robot2": robot2
}


Define points class

In [ ]:
class PointBuffer:
    def __init__(self, buffer_size):
        self.buffer = []
        self.buffer_size = buffer_size

    def update_buffer(self, new_x, new_y):
        if isinstance(new_x, (int, float)) and isinstance(new_y, (int, float)):
            self.buffer.append((new_x, new_y))
            if len(self.buffer) > self.buffer_size:
                self.buffer.pop(0)
        else:
            print("Invalid point: x or y is not a number")

    def calculate_median_point(self):
        if not self.buffer:
            return None, None

        x_coords, y_coords = zip(*self.buffer)
        median_x = np.median(x_coords)
        median_y = np.median(y_coords)

        return int(median_x), int(median_y)

Define Game class

In [ ]:
class Game:
    def __init__(self):
        self.attacking = False
        self.deffending = False
    
    def set_attacking(self, value: bool):
        self.attacking = value
        self.deffending = not value
    
    def set_deffending(self, value: bool):
        self.deffending = value
        self.attacking = not value



Define events in aseba

In [ ]:
%%run_aseba

onevent walk300
  motor.left.target = 300
  motor.right.target = 300
  
onevent walk200
  motor.left.target = 200
  motor.right.target = 200

onevent walk100
  motor.left.target = 100
  motor.right.target = 100

onevent walk50
  motor.left.target = 50
  motor.right.target = 50

onevent left_turn
  motor.left.target = 100
  motor.right.target = -100

onevent right_turn
  motor.left.target = -100
  motor.right.target = 100

onevent stop
  motor.left.target = 0
  motor.right.target = 0

In [ ]:
def move_to_point(angle_of_rotation, distance_to_point, rotation_threshold, distance_threshold):
    # print(angle_of_rotation)
    if ( abs(angle_of_rotation) > rotation_threshold) and (distance_to_point > distance_threshold):
        # print("inside 1", " ", angle_of_rotation)
        if(angle_of_rotation > 0):
            print("direita")
            send_event("right_turn")
        else:
            print("esquerda")
            send_event("left_turn")
    else:
        # print("inside 2")
        if(distance_to_point > distance_threshold) :
            print("andar")
            send_event("walk300")
        else:
            send_event("stop")

Variables

In [ ]:
COLOR_TOLERANCE: int = 20
DIST_BALL: int = 15
RT_TRESH: int = 20
DT_TRESH: int = 20
GOAL_DT_TRESH: int = 35


In [ ]:
prev_shoot_x = None
prev_shoot_y = None
shooting: bool = False
game: Game = Game.__init__()

Control random noise

In [ ]:
robot_front_buffer_size = 3
robot_back_buffer_size = 3
ball_buffer_size = 5
shooting_point_buffer_size = 20
goal1_buffer_size = 60
goal2_buffer_size = 60

robot_front_buffer = PointBuffer(robot_front_buffer_size)
robot_back_buffer = PointBuffer(robot_back_buffer_size)
ball_buffer = PointBuffer(ball_buffer_size)
shooting_point_buffer = PointBuffer(shooting_point_buffer_size)
goal1_buffer = PointBuffer(goal1_buffer_size)
goal2_buffer = PointBuffer(goal2_buffer_size)

In [ ]:
cap = cv2.VideoCapture(1)

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    matrix = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

    cv2.imshow('real_frame', frame)
    # Process each robot's front and back points
    for robot, details in robots.items():
        # Detect front points
        f_lower_bound = np.array(details['f_hsv'][0])
        f_upper_bound = np.array(details['f_hsv'][1])
        front_coordinates = utils.detect_colored_dots_in_rgb(image, f_lower_bound, f_upper_bound)
        robot_front_buffer.update_buffer(*utils.safe_get(front_coordinates, 0, [None, None]))

        # Mark front points on the matrix
        for coord in front_coordinates:
            matrix[coord[0], coord[1]] = details['front']
        
        # Detect back points
        b_lower_bound = np.array(details['b_hsv'][0])
        b_upper_bound = np.array(details['b_hsv'][1])
        back_coordinates = utils.detect_colored_dots_in_rgb(image, b_lower_bound, b_upper_bound)
        robot_back_buffer.update_buffer(*utils.safe_get(front_coordinates, 0, [None, None]))


        # Mark back points on the matrix
        for coord in back_coordinates:
            matrix[coord[0], coord[1]] = details['back']
        


    # Find the coordinates of goals and ball
    for feature, details in field.items():
        lower_bound = np.array(details['hsv'][0])
        upper_bound = np.array(details['hsv'][1])
        coordinates = utils.detect_colored_dots_in_rgb(image, lower_bound, upper_bound)
        
        match feature:
            case 'ball':
                ball_buffer.update_buffer(*utils.safe_get(coordinates, 0, [None, None]))
            case 'goal1':
                goal1_buffer.update_buffer(*utils.safe_get(coordinates, 0, [None, None]))
            case 'goal2':
                goal2_buffer.update_buffer(*utils.safe_get(coordinates, 0, [None, None]))
            case default:
                pass

        for coord in coordinates:
            matrix[coord[0], coord[1]] = details['center']
            # print(feature, coord[0], coord[1], details['center'])
    

    #___________________________________________________________TEST_PART______________________________________

    color_map = {
        # 0: (0, 0, 0),       # Black (assuming 0 is the background or no dot)
        1: (255, 0, 0),     # Red
        2: (0, 255, 0),     # Green
        3: (0, 0, 255),     # Blue
        4: (255, 255, 0),   # Cyan
        5: (255, 0, 255),   # Magenta
        6: (0, 255, 255),   # Yellow
        7: (255, 255, 255)  # White
    }

    # Create an empty frame with 3 color channels
    colored_frame = np.zeros((matrix.shape[0], matrix.shape[1], 3), dtype=np.uint8)

    # Draw colored circles in the frame based on matrix values
    for y in range(matrix.shape[0]):
        for x in range(matrix.shape[1]):
            value = matrix[y, x]
            if value in color_map:
                cv2.circle(colored_frame, (x, y), 10, color_map[value], 14)  # Circle with radius 10
    
    colored_fast_frame = np.zeros((matrix.shape[0], matrix.shape[1], 3), dtype=np.uint8)


    try:
        median_front = robot_front_buffer.calculate_median_point()
        median_back = robot_back_buffer.calculate_median_point()
        median_ball = ball_buffer.calculate_median_point()
        median_goal1 = goal1_buffer.calculate_median_point()
        median_goal2 = goal2_buffer.calculate_median_point()

        shoot_x_calc, shoot_y_calc = utils.ball_shooting_point(median_ball, median_goal2, DIST_BALL, False)
        shooting_point_buffer.update_buffer(shoot_x_calc, shoot_y_calc)
        median_shooting_point = shooting_point_buffer.calculate_median_point()
        



        cv2.line(colored_frame, median_shooting_point[::-1], median_goal2[::-1], (255, 255, 255))

        cv2.line(colored_frame, median_back[::-1], median_shooting_point[::-1], (0, 255, 255))

        r1x, r1y, r1orientation = utils.calculate_robot_position_and_orientation(matrix, median_front, median_back)
        angle_of_rotation_sp = utils.get_angle_of_rotation((r1x, r1y, r1orientation), median_shooting_point)
        # print(angle_of_rotation_sp)
        dist_shoot_sp = utils.get_distance_between_two_points((r1x, r1y), median_shooting_point)
        angle_of_rotation_defense = utils.get_angle_of_rotation((r1x, r1y, r1orientation), median_goal1)
        dist_defense_sp = utils.get_distance_between_two_points((r1x, r1y), median_goal1)
        print(angle_of_rotation_sp, dist_shoot_sp, shooting)
        center_field = utils.find_midpoint(median_goal1, median_goal2)

        if ball_fixed[1] < center_field[1]:
            if dist_shoot_sp > DT_TRESH and not(game.shooting):
                move_to_point(angle_of_rotation_sp, dist_shoot_sp, RT_TRESH, DT_TRESH)
            else:
                angle_of_rotation_ball = utils.get_angle_of_rotation((r1x, r1y, r1orientation), median_goal2)
                dist_shoot_ball = utils.get_distance_between_two_points((r1x, r1y), ball_fixed)
                move_to_point(angle_of_rotation_ball, dist_shoot_ball, RT_TRESH+10, DT_TRESH+10)
        else:
            if dist_defense_sp > GOAL_DT_TRESH and game.defending:
                move_to_point(angle_of_rotation_defense, dist_defense_sp, RT_TRESH, GOAL_DT_TRESH)

        if dist_shoot_sp < DT_TRESH and not(shooting):
            game.set_shooting(True)
            ball_fixed = median_ball
            print("switching")
        

            
    except:
        pass
    colored_frame = cv2.cvtColor(colored_frame, cv2.COLOR_RGB2BGR)
    # Display the frame
    cv2.imshow('Frame', colored_frame)

    colored_frame = cv2.cvtColor(colored_frame, cv2.COLOR_RGB2BGR)
    # Display the frame
    cv2.imshow('fast_frame', colored_fast_frame)

    # Wait for 'q' key to quit
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    #___________________________________________________________TEST_PART______________________________________
    

cv2.destroyAllWindows()

In [ ]:
cap.release()

In [ ]:
send_event("stop")